# Исследование надёжности заёмщиков

**Цель исследования**:
Имея статистику банка о платёжеспособности клиентов определить влияние семейного положения и количества детей на факт погашения кредита в срок.

**Основные вопросы исследования:**
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**Оглавление**
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)<br>
    2.1.[Обработка пропущенных значений](#null)<br>
    2.2.[Замена типов данных](#dtype)<br>
    2.3.[Обработка дубликатов](#duplicates)<br>
    2.4.[Лемматизация](#lemma)<br>
    2.5.[Категоризация данных](#category)<br>
3. [Анализ данных](#analysis)<br>
    3.1[Зависимость между наличием детей и возвратом кредита в срок](#children)<br>
    3.2[Зависимость между семейным положением и возвратом кредита в срок](#status)<br>
    3.3[Зависимость между уровнем дохода и возвратом кредита в срок](#income)<br>
    3.4[Зависимость между целью кредита возврат кредита в срок](#purpose)<br>
4. [Общий вывод](#conclusion)

## <a id="start">1 Открытие данных</a> 

In [1]:
# импорт библиотеки pandas
import pandas as pd
# библиотека pymystem3 для лемматизации
from pymystem3 import Mystem

In [2]:
# чтение файла с данными и сохранение в df
try:
    df = pd.read_csv('/datasets/data.csv')
    # получение общей информации о данных в таблице df
    df.info()
except:
    print('Файл не найден')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# вывод первых десяти строк
df.sample(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3521,0,-1955.828275,62,среднее,1,женат / замужем,0,F,сотрудник,0,192923.638532,на покупку своего автомобиля
8729,3,-5177.253827,38,среднее,1,женат / замужем,0,F,компаньон,0,101506.409391,свой автомобиль
17680,0,-5635.770767,48,среднее,1,женат / замужем,0,M,сотрудник,0,65707.073731,покупка жилой недвижимости


**Вывод**

В столбцах `days_employed` и `total_income` содержатся пропуски. Тип данных в столбцах соответствует типу значений. Пропуски являются случайными(MAR - Missing At Random): доход сильно коррелирует с занимаемой должностью и образованием, а стаж с возрастом. Поэтому не все клиенты указали свой стаж и доход.
***
**Названия столбцов и их описание**
* **children** — количество детей в семье
* **days_employed** — общий трудовой стаж в днях
* **dob_years** — возраст клиента в годах
* **education** — уровень образования клиента
* **education_id** — идентификатор уровня образования
* **family_status** — семейное положение
* **family_status_id** — идентификатор семейного положения
* **gender** — пол клиента
* **income_type** — тип занятости
* **debt** — имел ли задолженность по возврату кредитов
* **total_income** — ежемесячный доход
* **purpose** — цель получения кредита <br>
`Нарушений стиля в названиях столбцов не найдено.`
***
Столбец `days_employed` содержит артефакты: отрицательные значения; значения имеют вещественный тип данных. Лучше заменить на целочисленный;<br>
Столбец `education` содержит одинаковые слова, но в разном регистре;<br>
Столбец `total_income` содержит вещественный тип данных. Лучше заменить на целочисленный;<br>
Столбец `purpose` содержит похожие по смыслу строки, но написанные по-разному. Лучше выполнить категоризацию.

## <a id="preprocessing">Предобработка данных</a> 

### <a id="null">Обработка пропущенных значений</a> 

In [4]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Около 10% значений в столбцах `days_employed` и `total_income` содержат пропуски.<br>
Значения в столбце `days_employed` не влияют на исследование, но значения в столбце `total_income` необходимы для ответа на один из вопросов.<br>
Удалять эти строки нельзя, т.к. будет отсутствовать большой пласт данных, и результаты всего исследования будут некорректны.<br>
Можно оставить пропуски. В таком случае они будут влиять только на результат одного вопроса (зависимость между уровнем дохода и возвратом кредита в срок). Но как уже было сказано почти 10% данных будут отсутсвовать.<br>
Поэтому заменим пропуски в столбцах медианными значениями. Замена пропусков на средние значения не совсем корректна в виду того, что в столбцах могут присутствовать данные, значения которых сильно отличаются от остальных, а среднее значение этого не учитывает.<br>
Заполним пропуски в столбце с доходами медианным значением в зависимости от типа занятости - `income_type`, т.к. они имеют зависимость.<br>

In [5]:
# применим метод transform для замены пропусков в столбце с доходами
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

В столбце со стажем работы - `days_employed` присутствуют отрицательные значения. Прежде чем заменять пропуски медианой нужно исправить эти значения. Отрицательные значения заменим модулем числа.

In [6]:
#замена отрицательных значений модулем числа
df['days_employed'] = df['days_employed'].abs()
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Теперь можно рассчитать медиану в зависимости от столбца `dob_years`, т.к. стаж зависит от возраста.

In [7]:
# применим метод transform для замены пропусков в столбце со стажем
df['days_employed'] = df['days_employed'].fillna(df.groupby('dob_years')['days_employed'].transform('median'))
# подсчёт пропусков
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

С помощью методов *fillna* и *transform* мы избавились от пропусков.

### <a id="dtype">Замена типов данных</a>

В столбцах `days_employed` и `total_income` содержится вещественный тип данных. Такое представление данных неудобно и визуально тяжело воспринимается. Поэтому заменим тип данных в этих столбцах на целочисленный методом *astype()*. Данный метод позволяет выбрать необходимый тип данных.

In [8]:
df[['total_income', 'days_employed']] = df[['total_income', 'days_employed']].astype('int64')

In [9]:
# проверим измененный тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Замена типа данных прошла успешно.

### <a id="duplicates">Обработка дубликатов</a> 

Сначала преверим наличие явных строк-дубликатов.

In [10]:
#поиск явных строк-дубликатов
print(f'Явных строк-дубликатов: {df.duplicated().sum()}')

Явных строк-дубликатов: 54


Перед тем как удалить строки-дубликаты проверим столбцы на наличие ошибочных значений. Исправив эти значения, возможно появятся новые явные строки-дубликаты, после чего их можно будет удалить.

In [11]:
#Уникальные значения столбца children
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Столбец содержит отрицательные значения, что является артефактом. Заменим эти значения модулем числа. Также нужно проверить ячейки содержат значение 20, т.к. это подозрительно большое значение.

In [12]:
df['children'] = df['children'].abs()
print(df['children'].unique())
print(f'Явных строк-дубликатов: {df.duplicated().sum()}')

[ 1  0  3  2  4 20  5]
Явных строк-дубликатов: 54


In [13]:
df[df['children'] == 20]['children'].count()

76

76 семей с 20-ю детьми. Весьма маловероятно. Заменим эти значения на 2.

In [14]:
df.loc[df['children'] == 20, 'children'] = 2

In [15]:
#Уникальные значения столбца dob_years
df['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Здесь присутствует значение-артефакт: 0. Но оно не будет влиять на наличие явных строк-дубликатов.

In [16]:
#Уникальные значения столбца education
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В данном столбце есть повторяющиеся значения, но в разном регистре. Приведем все значения к нижнему регистру.

In [17]:
#приведем все значения в столбце education к нижнему регистру
df['education'] = df['education'].str.lower()
#поиск явных дубликатов
print(f'Явных строк-дубликатов: {df.duplicated().sum()}')

Явных строк-дубликатов: 71


In [18]:
#Уникальные значения столбца education_id
df['education_id'].unique()

array([0, 1, 2, 3, 4])

В этом столбце все в порядке.

In [19]:
#Уникальные значения столбца family_status
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В этом столбце все в порядке.

In [20]:
#Уникальные значения столбца family_status_id
df['family_status_id'].unique()

array([0, 1, 2, 3, 4])

В этом столбце все в порядке.

In [21]:
#Уникальные значения столбца gender
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Есть артефакт, но он не влияет на наличие явных строк-дубликатов.

In [22]:
#Уникальные значения столбца income_type
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В этом столбце все в порядке.

In [23]:
#Уникальные значения столбца debt
df['debt'].unique()

array([0, 1])

В этом столбце все в порядке.

In [24]:
#Уникальные значения столбца purpose
df['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

В этом столбце все в порядке.
***

Теперь можно удалять явные строки-дубликаты

In [25]:
#количество строк до удаления
print(f'Количество строк до удаления: {df.shape[0]}')
#удаляем явные дубликаты
df = df.drop_duplicates().reset_index(drop=True)
#проверяем количество оставшихся строк
print(f'Количество строк после удаления строк-дубликатов: {df.shape[0]}')

Количество строк до удаления: 21525
Количество строк после удаления строк-дубликатов: 21454


**Вывод**

Была найдена и удалена 71 строка-дубликат.

### <a id="lemma">Лемматизация</a>

Теперь выполним лемматизацию столбца с целями кредита - `purpose` для дальнейшей категоризации. Посмотрим на уникальные значения столбца с целями.

In [26]:
sorted(df['purpose'].unique())

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

Можно выделить ключевые слова: *автомобиль, недвижимость, образование, свадьба*. Лемматизация поможет привести слова к их словарной форме.Выполним лемматизацию с помощью библиотеки *pymystem3*. Создадим функцию, а затем применим ее к датафрейму через метод *apply()*.

In [27]:
m = Mystem() 

#напишем функцию, которая будет возвращать слово, если оно есть в строке после лемматизации
def purpose_lemma(row):
    lemmas = m.lemmatize(row['purpose'])
    if ('недвижимость' in lemmas) or ('жилье'in lemmas):
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
#вызовим функцию методом apply()
df['purpose_short'] = df.apply(purpose_lemma, axis=1)
#проверим результат
df[['purpose', 'purpose_short']].head(10)

,purpose,purpose_short
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
5,покупка жилья,недвижимость
6,операции с жильем,недвижимость
7,образование,образование
8,на проведение свадьбы,свадьба
9,покупка жилья для семьи,недвижимость


**Вывод**

Лемматизация позволила привести значения столбца с целями к словарной форме и выделить ключевые слова, которые были добавлены в новый столбец `purpose_short`.

### <a id="category">Категоризация данных</a>

Выполним категоризацию для тех данных, которые нужны для проверки гипотез. А именно:
* создадим словарь для столбца `family_status`
* проведем классификацию для столбца  `total_income`

Создадим словарь для столбцов `family_status` и `family_status_id`.

In [28]:
#создаем новый датафрейм из основного 
family_dict = df[['family_status', 'family_status_id']]
family_dict.head(5)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


In [29]:
#теперь удаляем дубликаты, сортируем таблицу и получаем словарь
family_dict = family_dict.drop_duplicates().reset_index(drop=True).sort_values(by='family_status_id')
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Чтобы выполнить категоризацию для столбца с доходами узнаем как распределяются доходы по квартилям.

In [30]:
df['total_income'].describe()

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

Получаем 4 категории с доходом, в каждой из которых 25% от общего числа людей:
* до 107623
* от 107623 до 142594
* от 142594 до 195820
* свыше 195820

Создадим функцию для распределения значений из столбца с доходом по группам, а затем применим метод *apply()* к новому столбцу `income_category`. 

In [31]:
def income_check(total_income):
    if total_income <= 107623:
        return 'до 107623'
    if total_income <= 142594:
        return 'от 107623 до 142594'
    if total_income <= 195820:
        return 'от 142594 до 195820'
    if total_income > 195820:
        return 'свыше 195820'
#применим функцию к столбцу children    
df['income_category'] = df['total_income'].apply(income_check)
df[['total_income', 'income_category']].head(10)

,total_income,income_category
0,253875,свыше 195820
1,112080,от 107623 до 142594
2,145885,от 142594 до 195820
3,267628,свыше 195820
4,158616,от 142594 до 195820
5,255763,свыше 195820
6,240525,свыше 195820
7,135823,от 107623 до 142594
8,95856,до 107623
9,144425,от 142594 до 195820


In [32]:
df['income_category'].value_counts()

от 107623 до 142594    5479
до 107623              5364
свыше 195820           5364
от 142594 до 195820    5247
Name: income_category, dtype: int64

**Вывод**

Была проведена категоризация, благодаря которой данные обрели структурированный вид. Теперь можно приступать к поиску ответов на вопросы исследования.

## <a id="analysis">Анализ данных</a>

### <a id="children">Зависимость между наличием детей и возвратом кредита в срок</a>

На данный вопрос можно ответить если создать сводную таблицу с данными из столбца `children` и `debt`.

In [33]:
#создадим сводную таблицу
debt_pivot_child = df.pivot_table(index='children', columns= 'debt', values='total_income', aggfunc=('count'), fill_value=0)
#создадим еще один столбец sum, чтобы увидеть общее количество людей в каждой группе
debt_pivot_child['sum'] = debt_pivot_child[0] + debt_pivot_child[1]
#выведем результат на экран
debt_pivot_child

debt,0,1,sum
children,,,
0,13028,1063,14091
1,4410,445,4855
2,1926,202,2128
3,303,27,330
4,37,4,41
5,9,0,9


Теперь найдем отношение людей с задолженность к общему количеству людей в каждой группе

In [34]:
debt_pivot_child['ratio'] = debt_pivot_child[1] / debt_pivot_child['sum']*100
debt_pivot_child

debt,0,1,sum,ratio
children,,,,
0,13028,1063,14091,7.543822
1,4410,445,4855,9.165808
2,1926,202,2128,9.492481
3,303,27,330,8.181818
4,37,4,41,9.756098
5,9,0,9,0.000000


**Вывод**

В зависимости от наличия детей задолженность имели :
* клиенты без детей - 7,5%;
* у клиента один ребенок - 9,2%;
* у клиента два ребенка - 9,5%;
* у клиента три ребенка - 8,2%;
* у клиента четыре ребенка - 9,8%;
***
**Клиенты с детьми чаше не возвращали деньги в срок. Это вполне логично, ведь при наличии детей увеличиваются расходы. Из тех, у кого есть дети чаше других имели задолженность те, у которых два или четыре ребенка.**

### <a id="status">Зависимость между семейным положением и возвратом кредита в срок</a>

На данный вопрос как и на следующие ответим таким же способом как и на первый вопрос.

In [35]:
debt_pivot_family = df.pivot_table(index='family_status_id', columns= 'debt', values='total_income', aggfunc=('count'))
debt_pivot_family['sum'] = debt_pivot_family[0] + debt_pivot_family[1]
debt_pivot_family['ratio'] = debt_pivot_family[1] / debt_pivot_family['sum']*100
display(debt_pivot_family)
print('Словарь: семейный статус\id')
family_dict

debt,0,1,sum,ratio
family_status_id,,,,
0,11408,931,12339,7.545182
1,3763,388,4151,9.347145
2,896,63,959,6.569343
3,1110,85,1195,7.112971
4,2536,274,2810,9.750890


Словарь: семейный статус\id


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


**Вывод**

Задолженность имели:
* люди в браке - 7,5% 
* люди в гражданскком браке - 9,3% 
* овдовевшие - 6,6% 
* в разводе - 7,1% 
* холостые - 9,8%
***
**Наиболее рискованные категории - люди в гражданском браке и холостые.**

### <a id="income">Зависимость между уровнем дохода и возвратом кредита в срок</a>

In [36]:
debt_pivot_income = df.pivot_table(index='income_category', columns= 'debt', values='total_income', aggfunc=('count'))
debt_pivot_income['sum'] = debt_pivot_income[0] + debt_pivot_income[1]
debt_pivot_income['ratio'] = debt_pivot_income[1] / debt_pivot_income['sum']*100
debt_pivot_income

debt,0,1,sum,ratio
income_category,,,,
до 107623,4937,427,5364,7.960477
от 107623 до 142594,4996,483,5479,8.815477
от 142594 до 195820,4799,448,5247,8.538212
свыше 195820,4981,383,5364,7.140194


**Вывод**

Задолженность имели люди с доходом:
* до 107623 - 8% 
* от 107623 до 142594 - 8,8% 
* от 142594 до 195820- 8,3% 
* свыше 195820 - 7,1% 
***
**Наиболее рискованные категории - люди с доходом от 107623 до 142594.**

### <a id="purpose">Зависимость между целью кредита возврат кредита в срок</a>

In [37]:
debt_pivot_purpose = df.pivot_table(index='purpose_short', columns= 'debt', values='total_income', aggfunc=('count'))
debt_pivot_purpose['sum'] = debt_pivot_purpose[0] + debt_pivot_purpose[1]
debt_pivot_purpose['ratio'] = debt_pivot_purpose[1] / debt_pivot_purpose['sum']*100
debt_pivot_purpose

debt,0,1,sum,ratio
purpose_short,,,,
автомобиль,3903,403,4306,9.359034
недвижимость,10029,782,10811,7.233373
образование,3643,370,4013,9.220035
свадьба,2138,186,2324,8.003442


**Вывод**

Задолженность в зависимости от целей кредита:
* автомобиль - 9,35%
* недвижимость - 7,23%
* образование - 9,22%
* свадьба - 8%
***
**Наиболее рискованные категории - автомобиль и образование. Возможно это связано с тем, что часто кредиты на подобные цели берут молодые люди с небольшим доходом.**

## <a id="conclusion">Общий вывод</a>

Были получены следующие результаты:
* клиенты с детьми чаше имели задолженность по кредиту, чем клиенты без детей. При этом чаще других не возвращали деньги в срок клиенты, у которых двое(9.5%) или четыре(9,8%);
* в зависимости от семейного положения наиболее рискованные категории - клиенты в гражданском браке(9,3%) и холостые(9,8%); наименее рискованная - овдовевшие(6,6%);
* в зависимости от дохода наиболее рискованная категория - клиенты с доходом от 107623 до 142594(8,8%); наименее рискованная - с доходом свыше 195820(7,1%);
* в зависимости от цели кредита наиболее рискованные категории - автомобиль(9,35%) и образование(9,22%); наименее рискованная - недвижимость(7,23%).